In [36]:
import requests
from bs4 import BeautifulSoup
import re

In [37]:
url = 'https://www.melon.com/chart/index.htm'
req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}
res = requests.get(url, headers=req_header_dict)
print(res.status_code)

200


In [38]:
if res.ok:
    html = res.text
    #html 전체 소스보기
    #print(html)
    soup = BeautifulSoup(html, 'html.parser')
    #html parsing
    #print(soup)  

In [39]:

#parsing한 html중 곡에대한 정보를 담음 태그만 조회!!
a_tags = soup.select("div#tb_list tr a[href*='playSong']")
print(len(soup.select("div#tb_list tr a[href*='playSong']")))  
#top100이니까 100이 나와야한다.
#print(a_tags)

100


In [40]:
# playSong이 있는 <a></a>를 enumerate하여 dict형식으로 만들어준다.
#Top 100곡의 정보를 저장할 list 선언
song_list = []
for idx, a_tag in enumerate(a_tags,1):
    #노래 1곡의 정보를 저장할 dict 선언 
    song_dict = {}
    #노래 제목 갖고오기 <a href="">노래제목</a>
    song_title = a_tag.text
    song_dict['song_title'] = song_title
    
    #print(song_dict)
    
    #a태그의 href 속성의 값을 추출하기 javascript:melon.play.playSong('1000002721',34535898);
    href_value = a_tag['href']

    #Song ID를 찾기 위한 정규표현식
    matched = re.search(r'(\d+)\);', href_value)
    #print(matched)
    if matched:
        song_id = matched.group(1) # group(0) : 34535898);  group(1) : 34535898
        song_dict['song_id'] = song_id
        #print(song_id)
        
        song_detail_url = f'https://www.melon.com/song/detail.htm?songId={song_id}' #곡정보의 url
        song_dict['song_detail_url'] = song_detail_url
        song_list.append(song_dict)

print(len(song_list))
print(song_list)

100
[{'song_title': '사랑은 늘 도망가', 'song_id': '34061322', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34061322'}, {'song_title': '취중고백', 'song_id': '34431086', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34431086'}, {'song_title': '호랑수월가', 'song_id': '34535898', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34535898'}, {'song_title': '이제 나만 믿어요', 'song_id': '32508053', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=32508053'}, {'song_title': 'Step Back', 'song_id': '34494149', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34494149'}, {'song_title': 'ELEVEN', 'song_id': '34349913', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=34349913'}, {'song_title': '그대라는 사치', 'song_id': '33652821', 'song_detail_url': 'https://www.melon.com/song/detail.htm?songId=33652821'}, {'song_title': '별빛 같은 나의 사랑아', 'song_id': '33337651', 'song_detail_url': 'https://www.melon.com/song/detail.htm?s

In [41]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.melon.com/chart/index.htm'
req_header_dict = {
    # 요청헤더 : 브라우저정보
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'
}
res = requests.get(url, headers=req_header_dict)
print(res.status_code)


200


In [48]:
song_detail_list = []
for idx, song in enumerate(song_list,1):
    #노래 1곡의 상세정보를 저장할 dict
    song_detail_dict = {}
    song_detail_url = song['song_detail_url']

    
    res = requests.get(song_detail_url, headers=req_header_dict)
    #print(res.status_code)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        #print(idx,song['song_title'])
        song_detail_dict['곡명'] = song['song_title']
        #노래 1곡의 상세정보에 '곡명'을 찾아 주었다. 
        #다음으로 상세정보 '가수'를 찾는다
        singer = soup.select("a[href*='goArtistDetail'] span")
        #print(singer)
        if singer:
            song_detail_dict['가수'] = singer[0].text # singer리스트에서 0번째 순서에 해당하는것을 text로 뽑으면 이름이다.
        #print(singer[0].text)
        
        #다음으로 div.meta에서 dd로 끝나는 값들을 넣어준다
        song_di = soup.select("div.meta dd")
        #print(song_di)
        if song_di:
            song_detail_dict['앨범'] = song_di[0].text
            song_detail_dict['발매일'] = song_di[1].text
            song_detail_dict['장르'] = song_di[2].text
        
        
        song_id = song['song_id'] #song_id = matched.group(1) 갖고 와서 곡별 고유 id를 이용하자
        like_url = f'https://www.melon.com/commonlike/getSongLike.json?contsIds={song_id}'
        like_res = requests.get(like_url, headers=req_header_dict)
        
        
        if like_res.ok:
           song_detail_dict['좋아요'] = like_res.json()['contsLike'][0]['SUMMCNT']
           #{'contsLike': [{'CONTSID': 34061322, 'LIKEYN': 'N', 'SUMMCNT': 124335}], 'httpDomain': 'http://www.melon.com', 'httpsDomain': 'https://www.melon.com', 'staticDomain': 'https://static.melon.co.kr'}
            #좋아요 수 추가 완료
        
        song_lyric = soup.select("div#d_video_summary")
        if song_lyric:
            song_detail_dict['가사'] = song_lyric[0].text
            pattern = re.compile(r'[\r\n\t]')
            # \r\n\t 특수문자를 ''(empty string)으로 대체(substitute)해라
            lyric = pattern.sub('', song_detail_dict['가사'].strip())
        else:
            lyric = ''
        song_detail_dict['가사'] = lyric
        
        song_detail_list.append(song_detail_dict)   
print(song_detail_list)

[]


In [43]:
import json

with open('songs.json','w',encoding='utf-8') as file:
    json.dump(song_detail_list, file)

In [49]:
## songs.json을 load 해보자 

with open('songs.json','r',encoding='utf-8') as file:
    songs_json = json.loads(file.read())
    
print(songs_json)

[]


In [50]:
import pandas as pd

song_df2 = pd.DataFrame(columns=['곡명','가수','앨범','발매일','좋아요','가사'])
for song_detail in song_detail_list:
    series_row = pd.Series(song_detail)
    song_df2 = song_df2.append(series_row, ignore_index=True)

In [51]:
song_df = pd.read_json('songs.json')
song_df.head()

""


In [53]:
#가수의 컬럼의 값을 선택하기

song_df['가수']

#가수의 컬럼의 값별로 row counting 
#장르이 컬럼의 값별로 row counting
song_df['가수'].value_counts
song_df['장르'].value_counts

#중복제거 가수 컬럼 값을 가져오기

song_df['가수'].unique



KeyError: '가수'

In [ ]:
### 특정 행과 열을 선택하기 

#가수가 방탄소년단인 행을 선택하기
temp = str(input()) # '방탄소년단' 입력 
song_df['가수'] == temp
song_df.loc[song_df['가수'] == temp]

#가수가 임영웅인 행을 선택하는 함수
def get_song(singer):
    song_df.loc[song_df['가수'] == singer]

get_song('임영웅')

# slicing : 인덱스가 0부터 5까지의 모든 행과 열을 선택하기
song_df.loc[:5]

# slicing : 인덱스가 0부터 10까지의 모든 행과 열을 선택하기, 1개의 행을 skip
song_df.loc[:10:2] #기본[0:10:1,:]

# slicing : 인덱스가 0부터 10까지의 모든 행과 곡명부터 장르까지의 열을 선택하기
song_df.loc[0:10,'곡명':'장르']

# slicing : 인덱스가 0부터 10까지의 모든 행과 곡명,가수, 발매일 까지의 열을 선택하기
song_df.loc[0:10,['곡명','가수','발매일']]

# 특정 열만 출력
def get_song(singer):
    return song_df.loc[song_df['가수'] == singer, '곡명':'좋아요']
get_song('임영웅')

#좋아요 건수가 평균보다 높은 행을 선택
like_avg = song_df['좋아요'].mean()
print(like_avg)

print('like max',song_df['좋아요'].max())
print('like min',song_df['좋아요'].min())
print('like median',song_df['좋아요'].median())

#좋아요가 가장많은 행 선택
like_max =song_df['좋아요'].max()
song_df.loc[song_df['좋아요'] == like_max]

#좋아요가 평균보다 높은 행 선택
like_avg = song_df['좋아요'].mean()
song_df.loc[song_df['좋아요'] >= like_avg]

#좋아요 건수가 평균보다 높은 행과 곡명,가수,좋아요 열을 선택하고 index를 reset해주자
like_avg = song_df['좋아요'].mean()
song_df.loc[song_df['좋아요'] >= like_avg, ['곡명','가수','좋아요']].reset_index(drop=True)

##좋아요 건수가 평균보다 높은 행과 곡명, 가수, 좋아요 열을 선택하고 좋아요 건수가 높은 순서대로 정렬하고, index reset하기
song_df.loc[song_df['좋아요'] >= like_avg, ['곡명','가수','좋아요']].sort_values(by='좋아요',ascending=False).reset_index(drop=True)